In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

In [3]:
from langchain_openai import ChatOpenAI
llm_llama70b = ChatOpenAI(
    base_url="https://api.together.xyz/v1", #https://api.together.xyz
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    temperature=0,)

In [4]:
llm_transformer_llama70b = LLMGraphTransformer(llm=llm_llama70b)

In [ ]:
loader = TextLoader(file_path="data/capitulo1.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [41]:
graph_documents = llm_transformer_llama70b.convert_to_graph_documents(documents)

In [42]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Úrsula', type='Person', properties={}), Node(id='Los Gitanos', type='Group', properties={}), Node(id='Melquíades', type='Person', properties={}), Node(id='La Población', type='Group', properties={})], relationships=[Relationship(source=Node(id='Úrsula', type='Person', properties={}), target=Node(id='La Población', type='Group', properties={}), type='INFLUENCED', properties={}), Relationship(source=Node(id='Los Gitanos', type='Group', properties={}), target=Node(id='Melquíades', type='Person', properties={}), type='TRANSFORMED', properties={}), Relationship(source=Node(id='Melquíades', type='Person', properties={}), target=Node(id='La Población', type='Group', properties={}), type='AMAZED', properties={})], source=Document(metadata={'source': 'parrafo_5.txt'}, page_content='Cuando volvieron los gitanos, Úrsula había predispuesto contra ellos a toda la población. Pero la curiosidad pudo más que el temor, porque aquella vez los gitanos recorrieron la aldea ha

In [43]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [5]:
def showGrath():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

In [6]:
showGrath()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [7]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [8]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()